### CNN-based 3D Skeleton Tennis Shot Recognition

Based on method from paper ['Skeleton-based Action Recognition with Convolutional Neural Networks'](https://arxiv.org/abs/1704.07595) by Li et al. (2017).

We use the 3D skeletal animation data from the ['THETIS'](http://thetis.image.ece.ntua.gr/) dataset for training. Each animation is represented as a `T x N` 3-channel image where `T` is the number of frames of the animation, `N` is the number of skeletal joints, and the 3 colour channels represent `x, y, z` coordinates of the joints.

We also use the motion data, represented in the same image format.

Load the skeleton and motion images training data.

In [120]:
import tensorflow as tf

skeleton_dir = "data/skeleton_images"
motion_dir = "data/motion_images"

img_height = 250
img_width = 15
batch_size = 32

skeleton_train_ds = tf.keras.utils.image_dataset_from_directory(
  skeleton_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

skeleton_val_ds = tf.keras.utils.image_dataset_from_directory(
  skeleton_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

motion_train_ds = tf.keras.utils.image_dataset_from_directory(
  motion_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

motion_val_ds = tf.keras.utils.image_dataset_from_directory(
  motion_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 621 files belonging to 24 classes.
Using 497 files for training.
Found 621 files belonging to 24 classes.
Using 124 files for validation.
Found 621 files belonging to 24 classes.
Using 497 files for training.
Found 621 files belonging to 24 classes.
Using 124 files for validation.


Create the CNN model.

In [121]:
from tensorflow.keras import layers, models

"""
skeleton_input = layers.Input(shape=(32,32,3))
skeleton_conv = layers.Conv2D(64, (3,3), activation='relu')(skeleton_input)
skeleton_pool = layers.MaxPooling2D((2,2))(skeleton_conv)

motion_input = layers.Input(shape=(32,32,3))
motion_conv = layers.Conv2D(64, (3,3), activation='relu')(motion_input)
motion_pool = layers.MaxPooling2D(2,2)(motion_conv)

concat = layers.Concatenate()([skeleton_pool, motion_pool])
flatten = layers.Flatten()(concat)
dense = layers.Dense(32, activation='relu')(flatten)
softmax = layers.Dense(24, activation='softmax')(dense)

model = tf.keras.Model(inputs=[skeleton_input, motion_input], outputs=[softmax])
"""

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(250,15,3)))
model.add(layers.MaxPooling2D((2,1)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,1)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(24, activation='softmax'))


Compile and train the model.

In [122]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  skeleton_train_ds,
  validation_data=skeleton_val_ds,
  epochs=1000
)

Epoch 1/1000
16/16 [==============================] - 3s 197ms/step - loss: 3.1898 - accuracy: 0.0423 - val_loss: 3.1832 - val_accuracy: 0.0403
Epoch 2/1000
16/16 [==============================] - 3s 182ms/step - loss: 3.1764 - accuracy: 0.0543 - val_loss: 3.1778 - val_accuracy: 0.0484
Epoch 3/1000
16/16 [==============================] - 3s 177ms/step - loss: 3.1457 - accuracy: 0.0785 - val_loss: 3.2011 - val_accuracy: 0.0484
Epoch 4/1000
16/16 [==============================] - 3s 180ms/step - loss: 3.0488 - accuracy: 0.0865 - val_loss: 3.2138 - val_accuracy: 0.0403
Epoch 5/1000
16/16 [==============================] - 3s 176ms/step - loss: 2.9103 - accuracy: 0.1288 - val_loss: 3.3423 - val_accuracy: 0.0887
Epoch 6/1000
16/16 [==============================] - 3s 176ms/step - loss: 2.7324 - accuracy: 0.1851 - val_loss: 3.4800 - val_accuracy: 0.0565
Epoch 7/1000
16/16 [==============================] - 3s 177ms/step - loss: 2.5226 - accuracy: 0.2495 - val_loss: 3.7508 - val_accuracy: